In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets
from IPython.display import display, clear_output
import time
from scipy.signal import firwin2, freqz, welch
import warnings
warnings.filterwarnings('ignore')

# Define the interactive widgets
f1_input = widgets.IntText(value=600, description='f1 (Hz):', continuous_update=False)
f2_input = widgets.IntText(value=1100, description='f2 (Hz):', continuous_update=False)
output_area = widgets.Output()

# Load an example signal
s = np.random.randn(5000)  # Example signal
Fs = 8192  # Sampling frequency

# Function to update plots based on filter frequencies
def update_plots(change):
    # Read the values from the input widgets
    f1, f2 = f1_input.value, f2_input.value

    if not (f1 < f2) or any(freq < 0 for freq in [f1, f2]) or any(freq > 3900 for freq in [f1, f2]):
        # Clear the output and display a warning message
        with output_area:
            output_area.clear_output(wait=True)
            warning_html = widgets.HTML(
                value="<div style='color: black; background-color: #ffffcc; padding: 10px; border-radius: 5px; font-size: 16px; text-align: center;'>" 
                      "<b>⚠️ Warning:</b> Frequency should not exceed 3900 and must be non-negative! Also, make sure f1 < f2!</div>",
                placeholder='',
                description='',
            )
            display(warning_html)
    else:
        # Proceed with the plot if all frequencies are valid
        with output_area:
            output_area.clear_output(wait=True)

            # Design the bandpass FIR filter
            freq11 = np.array([0, f1 * 0.95, f1 * 1.05, f2 * 0.95, f2 * 1.05, Fs / 2]) / (Fs / 2)
            gain = [0, 0, 1, 1, 0, 0]
            hbp_pm = firwin2(129, freq11, gain)

            # Compute the frequency response of the filter
            w, h = freqz(hbp_pm, worN=8000, fs=Fs)

            fig, axs = plt.subplots(1, 3, figsize=(14, 4))

            # Plot the impulse response
            axs[0].plot(hbp_pm, label='Bandpass Filter')
            axs[0].set_title('Impulse Response')
            axs[0].set_xlabel('Sample')
            axs[0].set_ylabel('Amplitude')
            axs[0].legend()
            axs[0].grid(True)

            # Plot the frequency response
            axs[1].plot(w, 20 * np.log10(abs(h)))
            axs[1].set_title('Frequency Response')
            axs[1].set_xlabel('Frequency [Hz]')
            axs[1].set_ylabel('Magnitude [dB]')
            axs[1].grid(True)

            # Convolve the signal with the filter coefficients
            s_pm = np.convolve(s, hbp_pm, 'same')

            # Plot the Power Spectral Density (PSD) of the filtered signal
            f, Pxx_spec = welch(s_pm, Fs, 'flattop', 1024, scaling='spectrum')
            axs[2].semilogy(f, np.sqrt(Pxx_spec))
            axs[2].set_xlabel('Frequency [Hz]')
            axs[2].set_ylabel('Linear spectrum [V RMS]')
            axs[2].set_title('Power Spectrum (Welch)')
            axs[2].grid(True)

            plt.tight_layout()
            plt.show()

# Attach the update_plots function to the frequency inputs
f1_input.observe(update_plots, names='value')
f2_input.observe(update_plots, names='value')

# Display widgets
input_ui = widgets.VBox([f1_input, f2_input])
ui = widgets.HBox([input_ui], layout=Layout(align_items='center'))

out = widgets.VBox([ui, output_area])
display(out)

# Call update_plots initially
update_plots(None)
